# Probability Estimation via Scoring


## 1. Logarithmic Scoring Rule

In [3]:
import numpy as np

y_true = np.array([1, 1, 0])

p_pred = np.array([0.8, 0.3, 0.6])

log_score = -np.mean(np.log(p_pred[y_true == 1]))
print(f"Log Score: {log_score:.4f}")

Log Score: 0.7136


## 2. Brier Score

In [5]:
from sklearn.metrics import brier_score_loss

y_true = np.array([1, 1, 0])

p_pred = np.array([0.8, 0.3, 0.6])

brier_score = brier_score_loss(y_true, p_pred)
print(f"Brier Score: {brier_score:.4f}")


Brier Score: 0.2967


## 3. Continuous Ranked Probability Score

In [8]:
import numpy as np
from properscoring import crps_ensemble

y_true = np.array([3.5])

predicted_ensemble = np.array([[3.0, 3.2, 3.4, 3.6, 3.8]])

crps_score = crps_ensemble(y_true, predicted_ensemble)
print(f"Continuous Ranked Probability Score: {crps_score.mean():.4f}")


Continuous Ranked Probability Score: 0.1000
